# **🔍 Project Title**

# 🔧 Fine-Tuning DeepSeek Coder 1.3B for Lawyer Chatbot using LoRA


# **🛠️ Environment Setup**

## ✅ Step 0: Install Required Libraries


In [3]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 494.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 46.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0
^C


# **📦 Load Pretrained Model**

## ✅ Step 1: Load Model & Tokenizer with 8bit Quantization

In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"

quant_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# **🎯 Apply LoRA Adapter**

## ✅ Step 2: Apply LoRA

In [2]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,572,864 || all params: 1,348,044,800 || trainable%: 0.1167


# **📁 Load and Format Dataset**

## ✅ Step 3: Load Dataset

In [3]:
from google.colab import files
uploaded = files.upload()


Saving lawyer_data.jsonl to lawyer_data.jsonl


In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="lawyer_data.jsonl")

def format_example(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    }

dataset = dataset.map(format_example)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# **✂️ Tokenization**

## ✅ Step 4: Tokenize

In [5]:

def tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    response = example["output"]

    input_enc = tokenizer(prompt + response, truncation=True, padding="max_length", max_length=512)
    label_enc = tokenizer(response, truncation=True, padding="max_length", max_length=512)

    label_enc["input_ids"] = [
        token if token != tokenizer.pad_token_id else -100 for token in label_enc["input_ids"]
    ]

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": label_enc["input_ids"]
    }

dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoConfig

# Load base model's config
base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"
config = AutoConfig.from_pretrained(base_model)

# Save it inside your fine-tuned model folder
config.save_pretrained("/content/deepseek_lawyer")  # or your path


# **🧠 Prepare Trainer**

## ✅ Step 5: Set Training Arguments

In [7]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="deepseek_lawyer",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)


/tmp/ipython-input-1122192795.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# **🚀 Train the Model**

## ✅ Step 6:  Train Model using LoRA

In [8]:
trainer.train()

Step,Training Loss
10,4.360300
20,4.455300
30,3.771100
40,3.306700
50,3.126300
60,2.666800
70,2.522800
80,2.377400
90,2.040300
100,1.978100


TrainOutput(global_step=750, training_loss=0.6103296391045053, metrics={'train_runtime': 417.7663, 'train_samples_per_second': 3.591, 'train_steps_per_second': 1.795, 'total_flos': 5907384631296000.0, 'train_loss': 0.6103296391045053, 'epoch': 3.0})

# **💾 Save the Model**

## ✅ Step 7: Save Fine-Tuned Model

In [9]:
model.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
config.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")

# **🧪 Inference / Test Chatbot**

## ✅ Step 8: Run Inference

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# Load base model from Hugging Face
base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

# Load your LoRA adapter
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/deepseek_lawyer")

# Inference function
def ask(question):
    prompt = f"### Instruction:\n{question}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = result.replace(prompt, "").strip()
    print("👨‍⚖️ Lawyer Assistant:\n", response)

# Example
ask("What is law?")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


👨‍⚖️ Lawyer Assistant:
 Law is the formal agreement between a lawyer and a court. It defines the rules and procedures of the law.


In [16]:
ask("what is criminal law?")

👨‍⚖️ Lawyer Assistant:
 Criminal law is the law in case of a crime. It defines the terms of the crime and the payment depending on the case.


In [17]:
ask("what is FIR?")

👨‍⚖️ Lawyer Assistant:
 FIR is short for Field-Interpolation. It is a method of interpolating missing data values in digital storage. It is commonly used in digital storage systems.
